<a href="https://colab.research.google.com/github/mgdixon/text2mol-team29/blob/main/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

# Install Cloud Storage FUSE.
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  53085      0 --:--:-- --:--:-- --:--:-- 53180
OK
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 10.4 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.0.1_amd64.deb ...
Unpacking gcsfuse (2.0.1) ...
Setting up g

In [2]:
# Mount a Cloud Storage bucket or location, without the gs:// prefix.
mount_path = "team29-text2mol"  # or a location like "my-bucket/path/to/mount"
local_path = f"/mnt/gs/{mount_path}"

!mkdir -p {local_path}
!gcsfuse --implicit-dirs {mount_path} {local_path}

{"timestamp":{"seconds":1714414812,"nanos":55706918},"severity":"INFO","message":"Start gcsfuse/2.0.1 (Go version go1.22.1) for app \"\" using mount point: /mnt/gs/team29-text2mol\n"}
{"timestamp":{"seconds":1714414812,"nanos":56474158},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":true,\"OnlyDir\":\"\",\"RenameDirLimit\":0,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier\":2,\"LocalFileCache\":false,\"TempDir\":\"\",\"ClientProtocol\":\"http1\",\"MaxConnsPerHost\":100,\"MaxIdleConnsPerHo

In [3]:
# Then you can access it like a local path.
!ls -lh {local_path}/MLP_outputs/

total 7.5G
drwxr-xr-x 1 root root    0 Apr 29 18:20 embeddings
-rw-r--r-- 1 root root 424M Apr  8 03:59 final_weights.40.pt
-rw-r--r-- 1 root root 424M Apr  8 00:46 weights_pretrained.00-1.60.pt
-rw-r--r-- 1 root root 424M Apr  8 00:51 weights_pretrained.01-0.65.pt
-rw-r--r-- 1 root root 424M Apr  8 00:57 weights_pretrained.02-0.64.pt
-rw-r--r-- 1 root root 424M Apr  8 01:03 weights_pretrained.03-0.55.pt
-rw-r--r-- 1 root root 424M Apr  8 01:08 weights_pretrained.04-0.48.pt
-rw-r--r-- 1 root root 424M Apr  8 01:14 weights_pretrained.05-0.40.pt
-rw-r--r-- 1 root root 424M Apr  8 01:19 weights_pretrained.06-0.33.pt
-rw-r--r-- 1 root root 424M Apr  8 01:29 weights_pretrained.08-0.32.pt
-rw-r--r-- 1 root root 424M Apr  8 01:35 weights_pretrained.09-0.28.pt
-rw-r--r-- 1 root root 424M Apr  8 01:45 weights_pretrained.11-0.27.pt
-rw-r--r-- 1 root root 424M Apr  8 01:50 weights_pretrained.12-0.27.pt
-rw-r--r-- 1 root root 424M Apr  8 02:05 weights_pretrained.15-0.21.pt
-rw-r--r-- 1 root root 4

In [4]:
import os
import shutil

import numpy as np

import matplotlib.pyplot as plt

import math

In [5]:
dir1 = local_path + "/GCN_output/embeddings/"
dir2 = local_path + "/GCN2/embeddings/"
dir3 = local_path + "/GCN3/embeddings/"
dir4 = local_path + "/MLP_outputs/embeddings/"
dir5 = local_path + "/MLP2/embeddings/"
dir6 = local_path + "/MLP3/embeddings/"


cids_train1 = np.load(dir1 + "cids_train.npy", allow_pickle=True)
cids_val1 = np.load(dir1 + "cids_val.npy", allow_pickle=True)
cids_test1 = np.load(dir1 + "cids_test.npy", allow_pickle=True)

chem_embeddings_train1 = np.load(dir1 + "chem_embeddings_train.npy")
chem_embeddings_val1 = np.load(dir1 + "chem_embeddings_val.npy")
chem_embeddings_test1 = np.load(dir1 + "chem_embeddings_test.npy")
text_embeddings_train1 = np.load(dir1 + "text_embeddings_train.npy")
text_embeddings_val1 = np.load(dir1 + "text_embeddings_val.npy")
text_embeddings_test1 = np.load(dir1 + "text_embeddings_test.npy")

cids_train2 = np.load(dir2 + "cids_train.npy", allow_pickle=True)
cids_val2 = np.load(dir2 + "cids_val.npy", allow_pickle=True)
cids_test2 = np.load(dir2 + "cids_test.npy", allow_pickle=True)

chem_embeddings_train2 = np.load(dir2 + "chem_embeddings_train.npy")
chem_embeddings_val2 = np.load(dir2 + "chem_embeddings_val.npy")
chem_embeddings_test2 = np.load(dir2 + "chem_embeddings_test.npy")
text_embeddings_train2 = np.load(dir2 + "text_embeddings_train.npy")
text_embeddings_val2 = np.load(dir2 + "text_embeddings_val.npy")
text_embeddings_test2 = np.load(dir2 + "text_embeddings_test.npy")

cids_train3 = np.load(dir3 + "cids_train.npy", allow_pickle=True)
cids_val3 = np.load(dir3 + "cids_val.npy", allow_pickle=True)
cids_test3 = np.load(dir3 + "cids_test.npy", allow_pickle=True)

chem_embeddings_train3 = np.load(dir3 + "chem_embeddings_train.npy")
chem_embeddings_val3 = np.load(dir3 + "chem_embeddings_val.npy")
chem_embeddings_test3 = np.load(dir3 + "chem_embeddings_test.npy")
text_embeddings_train3 = np.load(dir3 + "text_embeddings_train.npy")
text_embeddings_val3 = np.load(dir3 + "text_embeddings_val.npy")
text_embeddings_test3 = np.load(dir3 + "text_embeddings_test.npy")

cids_train4 = np.load(dir4 + "cids_train.npy", allow_pickle=True)
cids_val4 = np.load(dir4 + "cids_val.npy", allow_pickle=True)
cids_test4 = np.load(dir4 + "cids_test.npy", allow_pickle=True)

chem_embeddings_train4 = np.load(dir4 + "chem_embeddings_train.npy")
chem_embeddings_val4 = np.load(dir4 + "chem_embeddings_val.npy")
chem_embeddings_test4 = np.load(dir4 + "chem_embeddings_test.npy")
text_embeddings_train4 = np.load(dir4 + "text_embeddings_train.npy")
text_embeddings_val4 = np.load(dir4 + "text_embeddings_val.npy")
text_embeddings_test4 = np.load(dir4 + "text_embeddings_test.npy")

cids_train5 = np.load(dir5 + "cids_train.npy", allow_pickle=True)
cids_val5 = np.load(dir5 + "cids_val.npy", allow_pickle=True)
cids_test5 = np.load(dir5 + "cids_test.npy", allow_pickle=True)

chem_embeddings_train5 = np.load(dir5 + "chem_embeddings_train.npy")
chem_embeddings_val5 = np.load(dir5 + "chem_embeddings_val.npy")
chem_embeddings_test5 = np.load(dir5 + "chem_embeddings_test.npy")
text_embeddings_train5 = np.load(dir5 + "text_embeddings_train.npy")
text_embeddings_val5 = np.load(dir5 + "text_embeddings_val.npy")
text_embeddings_test5 = np.load(dir5 + "text_embeddings_test.npy")

cids_train6 = np.load(dir6 + "cids_train.npy", allow_pickle=True)
cids_val6 = np.load(dir6 + "cids_val.npy", allow_pickle=True)
cids_test6 = np.load(dir6 + "cids_test.npy", allow_pickle=True)

chem_embeddings_train6 = np.load(dir6 + "chem_embeddings_train.npy")
chem_embeddings_val6 = np.load(dir6 + "chem_embeddings_val.npy")
chem_embeddings_test6 = np.load(dir6 + "chem_embeddings_test.npy")
text_embeddings_train6 = np.load(dir6 + "text_embeddings_train.npy")
text_embeddings_val6 = np.load(dir6 + "text_embeddings_val.npy")
text_embeddings_test6 = np.load(dir6 + "text_embeddings_test.npy")

#Reorder (this is very important):
tmp = cids_train2.tolist()
indexes = [tmp.index(i) for i in cids_train1]
tmp = cids_val2.tolist()
indexes_val = [tmp.index(i) for i in cids_val1]
tmp = cids_test2.tolist()
indexes_test = [tmp.index(i) for i in cids_test1]

cids_train2 = cids_train2[indexes]
cids_val2 = cids_val2[indexes_val]
cids_test2 = cids_test2[indexes_test]

chem_embeddings_train2 = chem_embeddings_train2[indexes]
text_embeddings_train2 = text_embeddings_train2[indexes]
chem_embeddings_val2 = chem_embeddings_val2[indexes_val]
text_embeddings_val2 = text_embeddings_val2[indexes_val]
chem_embeddings_test2 = chem_embeddings_test2[indexes_test]
text_embeddings_test2 = text_embeddings_test2[indexes_test]


tmp = cids_train3.tolist()
indexes = [tmp.index(i) for i in cids_train1]
tmp = cids_val3.tolist()
indexes_val = [tmp.index(i) for i in cids_val1]
tmp = cids_test3.tolist()
indexes_test = [tmp.index(i) for i in cids_test1]

cids_train3 = cids_train3[indexes]
cids_val3 = cids_val3[indexes_val]
cids_test3 = cids_test3[indexes_test]

chem_embeddings_train3 = chem_embeddings_train3[indexes]
text_embeddings_train3 = text_embeddings_train3[indexes]
chem_embeddings_val3 = chem_embeddings_val3[indexes_val]
text_embeddings_val3 = text_embeddings_val3[indexes_val]
chem_embeddings_test3 = chem_embeddings_test3[indexes_test]
text_embeddings_test3 = text_embeddings_test3[indexes_test]


tmp = cids_train4.tolist()
indexes = [tmp.index(i) for i in cids_train1]
tmp = cids_val4.tolist()
indexes_val = [tmp.index(i) for i in cids_val1]
tmp = cids_test4.tolist()
indexes_test = [tmp.index(i) for i in cids_test1]

cids_train4 = cids_train4[indexes]
cids_val4 = cids_val4[indexes_val]
cids_test4 = cids_test4[indexes_test]

chem_embeddings_train4 = chem_embeddings_train4[indexes]
text_embeddings_train4 = text_embeddings_train4[indexes]
chem_embeddings_val4 = chem_embeddings_val4[indexes_val]
text_embeddings_val4 = text_embeddings_val4[indexes_val]
chem_embeddings_test4 = chem_embeddings_test4[indexes_test]
text_embeddings_test4 = text_embeddings_test4[indexes_test]


tmp = cids_train5.tolist()
indexes = [tmp.index(i) for i in cids_train1]
tmp = cids_val5.tolist()
indexes_val = [tmp.index(i) for i in cids_val1]
tmp = cids_test5.tolist()
indexes_test = [tmp.index(i) for i in cids_test1]

cids_train5 = cids_train5[indexes]
cids_val5 = cids_val5[indexes_val]
cids_test5 = cids_test5[indexes_test]

chem_embeddings_train5 = chem_embeddings_train5[indexes]
text_embeddings_train5 = text_embeddings_train5[indexes]
chem_embeddings_val5 = chem_embeddings_val5[indexes_val]
text_embeddings_val5 = text_embeddings_val5[indexes_val]
chem_embeddings_test5 = chem_embeddings_test5[indexes_test]
text_embeddings_test5 = text_embeddings_test5[indexes_test]


tmp = cids_train6.tolist()
indexes = [tmp.index(i) for i in cids_train1]
tmp = cids_val6.tolist()
indexes_val = [tmp.index(i) for i in cids_val1]
tmp = cids_test6.tolist()
indexes_test = [tmp.index(i) for i in cids_test1]

cids_train6 = cids_train6[indexes]
cids_val6 = cids_val6[indexes_val]
cids_test6 = cids_test6[indexes_test]

chem_embeddings_train6 = chem_embeddings_train6[indexes]
text_embeddings_train6 = text_embeddings_train6[indexes]
chem_embeddings_val6 = chem_embeddings_val6[indexes_val]
text_embeddings_val6 = text_embeddings_val6[indexes_val]
chem_embeddings_test6 = chem_embeddings_test6[indexes_test]
text_embeddings_test6 = text_embeddings_test6[indexes_test]


all_chem_embbedings1 = np.concatenate((chem_embeddings_train1, chem_embeddings_val1, chem_embeddings_test1), axis = 0)
all_chem_embbedings2 = np.concatenate((chem_embeddings_train2, chem_embeddings_val2, chem_embeddings_test2), axis = 0)
all_chem_embbedings3 = np.concatenate((chem_embeddings_train3, chem_embeddings_val3, chem_embeddings_test3), axis = 0)
all_chem_embbedings4 = np.concatenate((chem_embeddings_train4, chem_embeddings_val4, chem_embeddings_test4), axis = 0)
all_chem_embbedings5 = np.concatenate((chem_embeddings_train5, chem_embeddings_val5, chem_embeddings_test5), axis = 0)
all_chem_embbedings6 = np.concatenate((chem_embeddings_train6, chem_embeddings_val6, chem_embeddings_test6), axis = 0)

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def memory_efficient_similarity_matrix_custom(func, embedding1, embedding2, chunk_size = 1000):
    rows = embedding1.shape[0]

    num_chunks = int(np.ceil(rows / chunk_size))

    for i in range(num_chunks):
        end_chunk = (i+1)*(chunk_size) if (i+1)*(chunk_size) < rows else rows #account for smaller chunk at end...
        yield func(embedding1[i*chunk_size:end_chunk,:], embedding2)


text_chem_cos1 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_train1, all_chem_embbedings1)
text_chem_cos_val1 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_val1, all_chem_embbedings1)
text_chem_cos_test1 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_test1, all_chem_embbedings1)

text_chem_cos2 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_train2, all_chem_embbedings2)
text_chem_cos_val2 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_val2, all_chem_embbedings2)
text_chem_cos_test2 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_test2, all_chem_embbedings2)

text_chem_cos3 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_train3, all_chem_embbedings3)
text_chem_cos_val3 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_val3, all_chem_embbedings3)
text_chem_cos_test3 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_test3, all_chem_embbedings3)

text_chem_cos4 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_train4, all_chem_embbedings4)
text_chem_cos_val4 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_val4, all_chem_embbedings4)
text_chem_cos_test4 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_test4, all_chem_embbedings4)

text_chem_cos5 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_train5, all_chem_embbedings5)
text_chem_cos_val5 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_val5, all_chem_embbedings5)
text_chem_cos_test5 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_test5, all_chem_embbedings5)

text_chem_cos6 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_train6, all_chem_embbedings6)
text_chem_cos_val6 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_val6, all_chem_embbedings6)
text_chem_cos_test6 = memory_efficient_similarity_matrix_custom(cosine_similarity, text_embeddings_test6, all_chem_embbedings6)


In [7]:
n_train = len(cids_train1)
n_val = len(cids_val2)
n_test = len(cids_test1)
n = n_train + n_val + n_test

offset_val = n_train
offset_test = n_train + n_val

cids_all = np.concatenate((cids_train1, cids_val1, cids_test1), axis = 0)

In [8]:
tr_ranks_avg = np.zeros((n_train, n))

In [9]:
val_avg_ranks = np.zeros((n_val, n))
test_avg_ranks = np.zeros((n_test, n))

In [10]:
#For space 1:


tr_correct1 = np.zeros(len(cids_train1))

hits_at_one = 0
hits_at_ten = 0
hits_at_100 = 0
hits_at_500 = 0
hits_at_1000 = 0
ranks1 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos1):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs) #rank is actually double argsort.

        tr_ranks_avg[j,:] = tr_ranks_avg[j,:] + ranks


        tr_correct1[j] = ranks[j] + 1
        rank = ranks[j] + 1
        ranks1.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "train processed.")

ranks1 = np.array(ranks1)

print()
print("Training Mean rank:", np.mean(ranks1))
print("Hits at 1:", np.mean(ranks1 <= 1))
print("Hits at 10:", np.mean(ranks1 <= 10))
print("Hits at 100:", np.mean(ranks1 <= 100))
print("Hits at 500:", np.mean(ranks1 <= 500))
print("Hits at 1000:", np.mean(ranks1 <= 1000))

print("Trainng MRR:", np.mean(1/np.array(ranks1)))

1000 train processed.
2000 train processed.
3000 train processed.
4000 train processed.
5000 train processed.
6000 train processed.
7000 train processed.
8000 train processed.
9000 train processed.
10000 train processed.
11000 train processed.
12000 train processed.
13000 train processed.
14000 train processed.
15000 train processed.
16000 train processed.
17000 train processed.
18000 train processed.
19000 train processed.
20000 train processed.
21000 train processed.
22000 train processed.
23000 train processed.
24000 train processed.
25000 train processed.
26000 train processed.

Training Mean rank: 3.773401999394123
Hits at 1: 0.43077855195395337
Hits at 10: 0.9351711602544683
Hits at 100: 0.9992047864283551
Hits at 500: 1.0
Hits at 1000: 1.0
Trainng MRR: 0.6027827503823702


In [11]:

ranks_val1 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_val1):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        val_avg_ranks[j,:] = val_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_val] + 1
        ranks_val1.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "val processed.")


ranks_val1 = np.array(ranks_val1)

print()
print("Val Mean rank:", np.mean(ranks_val1))
print("Hits at 1:", np.mean(ranks_val1 <= 1))
print("Hits at 10:", np.mean(ranks_val1 <= 10))
print("Hits at 100:", np.mean(ranks_val1 <= 100))
print("Hits at 500:", np.mean(ranks_val1 <= 500))
print("Hits at 1000:", np.mean(ranks_val1 <= 1000))

print("Validation MRR:", np.mean(1/ranks_val1))

1000 val processed.
2000 val processed.
3000 val processed.

Val Mean rank: 25.33292941532869
Hits at 1: 0.33262647682520446
Hits at 10: 0.8270221145107544
Hits at 100: 0.966373826113299
Hits at 500: 0.9927294759163889
Hits at 1000: 0.9954559224477431
Validation MRR: 0.49462557204654617


In [12]:


ranks_test1 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_test1):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        test_avg_ranks[j,:] = test_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_test] + 1
        ranks_test1.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "test processed.")

ranks_test1 = np.array(ranks_test1)

print()
print("Test Mean rank:", np.mean(ranks_test1))
print("Hits at 1:", np.mean(ranks_test1 <= 1))
print("Hits at 10:", np.mean(ranks_test1 <= 10))
print("Hits at 100:", np.mean(ranks_test1 <= 100))
print("Hits at 500:", np.mean(ranks_test1 <= 500))
print("Hits at 1000:", np.mean(ranks_test1 <= 1000))

print("Test MRR:", np.mean(1/ranks_test1))

1000 test processed.
2000 test processed.
3000 test processed.

Test Mean rank: 26.569221448046047
Hits at 1: 0.32838533777643136
Hits at 10: 0.8273250530142381
Hits at 100: 0.9712208421690397
Hits at 500: 0.9921235989094214
Hits at 1000: 0.9966676764616783
Test MRR: 0.4929061871869085


In [13]:
#For space 2:

ranks2 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos2):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        tr_ranks_avg[j,:] = tr_ranks_avg[j,:] + ranks

        rank = ranks[j] + 1
        ranks2.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "train processed.")

ranks2 = np.array(ranks2)

print()
print("Training Mean rank:", np.mean(ranks2))
print("Hits at 1:", np.mean(ranks2 <= 1))
print("Hits at 10:", np.mean(ranks2 <= 10))
print("Hits at 100:", np.mean(ranks2 <= 100))
print("Hits at 500:", np.mean(ranks2 <= 500))
print("Hits at 1000:", np.mean(ranks2 <= 1000))

print("Training MRR:", np.mean(1/ranks2))

1000 train processed.
2000 train processed.
3000 train processed.
4000 train processed.
5000 train processed.
6000 train processed.
7000 train processed.
8000 train processed.
9000 train processed.
10000 train processed.
11000 train processed.
12000 train processed.
13000 train processed.
14000 train processed.
15000 train processed.
16000 train processed.
17000 train processed.
18000 train processed.
19000 train processed.
20000 train processed.
21000 train processed.
22000 train processed.
23000 train processed.
24000 train processed.
25000 train processed.
26000 train processed.

Training Mean rank: 3.777794607694638
Hits at 1: 0.4286579824295668
Hits at 10: 0.9322932444713723
Hits at 100: 0.9995834595577098
Hits at 500: 1.0
Hits at 1000: 1.0
Training MRR: 0.5998095560970966


In [14]:

ranks_val2 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_val2):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        val_avg_ranks[j,:] = val_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_val] + 1
        ranks_val2.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "val processed.")


ranks_val2 = np.array(ranks_val2)

print()
print("Val Mean rank:", np.mean(ranks_val2))
print("Hits at 1:", np.mean(ranks_val2 <= 1))
print("Hits at 10:", np.mean(ranks_val2 <= 10))
print("Hits at 100:", np.mean(ranks_val2 <= 100))
print("Hits at 500:", np.mean(ranks_val2 <= 500))
print("Hits at 1000:", np.mean(ranks_val2 <= 1000))

print("Validation MRR:", np.mean(1/ranks_val2))

1000 val processed.
2000 val processed.
3000 val processed.

Val Mean rank: 27.55528627688579
Hits at 1: 0.3238412602241745
Hits at 10: 0.8124810663435322
Hits at 100: 0.9706149651620721
Hits at 500: 0.9921235989094214
Hits at 1000: 0.9972735534686459
Validation MRR: 0.48576901772929704


In [15]:

ranks_test2 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_test2):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        test_avg_ranks[j,:] = test_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_test] + 1
        ranks_test2.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "test processed.")

ranks_test2 = np.array(ranks_test2)

print()
print("Test Mean rank:", np.mean(ranks_test2))
print("Hits at 1:", np.mean(ranks_test2 <= 1))
print("Hits at 10:", np.mean(ranks_test2 <= 10))
print("Hits at 100:", np.mean(ranks_test2 <= 100))
print("Hits at 500:", np.mean(ranks_test2 <= 500))
print("Hits at 1000:", np.mean(ranks_test2 <= 1000))

print("Test MRR:", np.mean(1/ranks_test2))

1000 test processed.
2000 test processed.
3000 test processed.

Test Mean rank: 30.252953650408966
Hits at 1: 0.33292941532868825
Hits at 10: 0.8276279915177219
Hits at 100: 0.97364435019691
Hits at 500: 0.9945471069372918
Hits at 1000: 0.9966676764616783
Test MRR: 0.49152052959419673


In [16]:
#For space 3:

ranks3 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos3):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        tr_ranks_avg[j,:] = tr_ranks_avg[j,:] + ranks

        rank = ranks[j] + 1
        ranks3.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "train processed.")

ranks3 = np.array(ranks3)

print()
print("Training Mean rank:", np.mean(ranks3))
print("Hits at 1:", np.mean(ranks3 <= 1))
print("Hits at 10:", np.mean(ranks3 <= 10))
print("Hits at 100:", np.mean(ranks3 <= 100))
print("Hits at 500:", np.mean(ranks3 <= 500))
print("Hits at 1000:", np.mean(ranks3 <= 1000))

print("Training MRR:", np.mean(1/ranks3))

1000 train processed.
2000 train processed.
3000 train processed.
4000 train processed.
5000 train processed.
6000 train processed.
7000 train processed.
8000 train processed.
9000 train processed.
10000 train processed.
11000 train processed.
12000 train processed.
13000 train processed.
14000 train processed.
15000 train processed.
16000 train processed.
17000 train processed.
18000 train processed.
19000 train processed.
20000 train processed.
21000 train processed.
22000 train processed.
23000 train processed.
24000 train processed.
25000 train processed.
26000 train processed.

Training Mean rank: 3.7033096031505606
Hits at 1: 0.4342244774310815
Hits at 10: 0.9346410178733717
Hits at 100: 0.9996213268706453
Hits at 500: 1.0
Hits at 1000: 1.0
Training MRR: 0.6054544374568394


In [17]:

ranks_val3 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_val3):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        val_avg_ranks[j,:] = val_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_val] + 1
        ranks_val3.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "val processed.")


ranks_val3 = np.array(ranks_val3)

print()
print("Val Mean rank:", np.mean(ranks_val3))
print("Hits at 1:", np.mean(ranks_val3 <= 1))
print("Hits at 10:", np.mean(ranks_val3 <= 10))
print("Hits at 100:", np.mean(ranks_val3 <= 100))
print("Hits at 500:", np.mean(ranks_val3 <= 500))
print("Hits at 1000:", np.mean(ranks_val3 <= 1000))

print("Validation MRR:", np.mean(1/ranks_val3))

1000 val processed.
2000 val processed.
3000 val processed.

Val Mean rank: 24.92275068161163
Hits at 1: 0.33020296879733413
Hits at 10: 0.8242956679794002
Hits at 100: 0.9700090881551046
Hits at 500: 0.9918206604059376
Hits at 1000: 0.9957588609512269
Validation MRR: 0.4926737813962398


In [18]:
tmp = cids_all[np.argsort(val_avg_ranks[1396,:])]
print(tmp)
print(np.where(tmp == '45359507')[0][0] + 1)

['6106' '439524' '857' ... '134160309' '134160310' '134160311']
28483


In [19]:

ranks_test3 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_test3):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        test_avg_ranks[j,:] = test_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_test] + 1
        ranks_test3.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "test processed.")

ranks_test3 = np.array(ranks_test3)

print()
print("Test Mean rank:", np.mean(ranks_test3))
print("Hits at 1:", np.mean(ranks_test3 <= 1))
print("Hits at 10:", np.mean(ranks_test3 <= 10))
print("Hits at 100:", np.mean(ranks_test3 <= 100))
print("Hits at 500:", np.mean(ranks_test3 <= 500))
print("Hits at 1000:", np.mean(ranks_test3 <= 1000))

print("Test MRR:", np.mean(1/ranks_test3))

1000 test processed.
2000 test processed.
3000 test processed.

Test Mean rank: 29.471978188427748
Hits at 1: 0.34262344744016965
Hits at 10: 0.8297485610421085
Hits at 100: 0.9742502272038777
Hits at 500: 0.9933353529233565
Hits at 1000: 0.9963647379581945
Test MRR: 0.5037819885980604


In [20]:
#For space 4:

ranks4 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos4):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        tr_ranks_avg[j,:] = tr_ranks_avg[j,:] + ranks

        rank = ranks[j] + 1
        ranks4.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "train processed.")

ranks4 = np.array(ranks4)

print()
print("Training Mean rank:", np.mean(ranks4))
print("Hits at 1:", np.mean(ranks4 <= 1))
print("Hits at 10:", np.mean(ranks4 <= 10))
print("Hits at 100:", np.mean(ranks4 <= 100))
print("Hits at 500:", np.mean(ranks4 <= 500))
print("Hits at 1000:", np.mean(ranks4 <= 1000))

print("Training MRR:", np.mean(1/ranks4))

1000 train processed.
2000 train processed.
3000 train processed.
4000 train processed.
5000 train processed.
6000 train processed.
7000 train processed.
8000 train processed.
9000 train processed.
10000 train processed.
11000 train processed.
12000 train processed.
13000 train processed.
14000 train processed.
15000 train processed.
16000 train processed.
17000 train processed.
18000 train processed.
19000 train processed.
20000 train processed.
21000 train processed.
22000 train processed.
23000 train processed.
24000 train processed.
25000 train processed.
26000 train processed.

Training Mean rank: 3.6195092396243562
Hits at 1: 0.4310436231445017
Hits at 10: 0.9364586488942744
Hits at 100: 0.9999621326870646
Hits at 500: 1.0
Hits at 1000: 1.0
Training MRR: 0.6040432238560232


In [21]:

ranks_val4 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_val4):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        val_avg_ranks[j,:] = val_avg_ranks[j,:] + ranks


        rank = ranks[j+offset_val] + 1
        ranks_val4.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "val processed.")


ranks_val4 = np.array(ranks_val4)

print()
print("Val Mean rank:", np.mean(ranks_val4))
print("Hits at 1:", np.mean(ranks_val4 <= 1))
print("Hits at 10:", np.mean(ranks_val4 <= 10))
print("Hits at 100:", np.mean(ranks_val4 <= 100))
print("Hits at 500:", np.mean(ranks_val4 <= 500))
print("Hits at 1000:", np.mean(ranks_val4 <= 1000))

print("Validation MRR:", np.mean(1/ranks_val4))

1000 val processed.
2000 val processed.
3000 val processed.

Val Mean rank: 32.12450772493184
Hits at 1: 0.33959406240533174
Hits at 10: 0.8309603150560436
Hits at 100: 0.9669797031202666
Hits at 500: 0.9909118448954862
Hits at 1000: 0.9930324144198728
Validation MRR: 0.5030397550109146


In [22]:

ranks_test4 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_test4):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        test_avg_ranks[j,:] = test_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_test] + 1
        ranks_test4.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "test processed.")

ranks_test4 = np.array(ranks_test4)

print()
print("Test Mean rank:", np.mean(ranks_test4))
print("Hits at 1:", np.mean(ranks_test4 <= 1))
print("Hits at 10:", np.mean(ranks_test4 <= 10))
print("Hits at 100:", np.mean(ranks_test4 <= 100))
print("Hits at 500:", np.mean(ranks_test4 <= 500))
print("Hits at 1000:", np.mean(ranks_test4 <= 1000))

print("Test MRR:", np.mean(1/ranks_test4))

1000 test processed.
2000 test processed.
3000 test processed.

Test Mean rank: 24.218418661011814
Hits at 1: 0.34989397152378066
Hits at 10: 0.8367161466222357
Hits at 100: 0.9760678582247804
Hits at 500: 0.9933353529233565
Hits at 1000: 0.9966676764616783
Test MRR: 0.5110214842104321


In [23]:
#For space 5:

ranks5 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos5):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        tr_ranks_avg[j,:] = tr_ranks_avg[j,:] + ranks

        rank = ranks[j] + 1
        ranks5.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "train processed.")

ranks5 = np.array(ranks5)

print()
print("Training Mean rank:", np.mean(ranks5))
print("Hits at 1:", np.mean(ranks5 <= 1))
print("Hits at 10:", np.mean(ranks5 <= 10))
print("Hits at 100:", np.mean(ranks5 <= 100))
print("Hits at 500:", np.mean(ranks5 <= 500))
print("Hits at 1000:", np.mean(ranks5 <= 1000))

print("Training MRR:", np.mean(1/ranks5))

1000 train processed.
2000 train processed.
3000 train processed.
4000 train processed.
5000 train processed.
6000 train processed.
7000 train processed.
8000 train processed.
9000 train processed.
10000 train processed.
11000 train processed.
12000 train processed.
13000 train processed.
14000 train processed.
15000 train processed.
16000 train processed.
17000 train processed.
18000 train processed.
19000 train processed.
20000 train processed.
21000 train processed.
22000 train processed.
23000 train processed.
24000 train processed.
25000 train processed.
26000 train processed.

Training Mean rank: 3.6427597697667373
Hits at 1: 0.43661011814601636
Hits at 10: 0.9365722508330809
Hits at 100: 1.0
Hits at 500: 1.0
Hits at 1000: 1.0
Training MRR: 0.6067333791962036


In [24]:

ranks_val5 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_val5):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        val_avg_ranks[j,:] = val_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_val] + 1
        ranks_val5.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "val processed.")


ranks_val5 = np.array(ranks_val5)

print()
print("Val Mean rank:", np.mean(ranks_val5))
print("Hits at 1:", np.mean(ranks_val5 <= 1))
print("Hits at 10:", np.mean(ranks_val5 <= 10))
print("Hits at 100:", np.mean(ranks_val5 <= 100))
print("Hits at 500:", np.mean(ranks_val5 <= 500))
print("Hits at 1000:", np.mean(ranks_val5 <= 1000))

print("Validation MRR:", np.mean(1/ranks_val5))

1000 val processed.
2000 val processed.
3000 val processed.

Val Mean rank: 28.23962435625568
Hits at 1: 0.3286882762799152
Hits at 10: 0.8358073311117843
Hits at 100: 0.9697061496516207
Hits at 500: 0.9924265374129052
Hits at 1000: 0.9954559224477431
Validation MRR: 0.49285549439178195


In [25]:

ranks_test5 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_test5):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        test_avg_ranks[j,:] = test_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_test] + 1
        ranks_test5.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "test processed.")

ranks_test5 = np.array(ranks_test5)

print()
print("Test Mean rank:", np.mean(ranks_test5))
print("Hits at 1:", np.mean(ranks_test5 <= 1))
print("Hits at 10:", np.mean(ranks_test5 <= 10))
print("Hits at 100:", np.mean(ranks_test5 <= 100))
print("Hits at 500:", np.mean(ranks_test5 <= 500))
print("Hits at 1000:", np.mean(ranks_test5 <= 1000))

print("Test MRR:", np.mean(1/ranks_test5))

1000 test processed.
2000 test processed.
3000 test processed.

Test Mean rank: 27.697061496516206
Hits at 1: 0.34656164798545896
Hits at 10: 0.8339897000908816
Hits at 100: 0.9745531657073614
Hits at 500: 0.9930324144198728
Hits at 1000: 0.9960617994547107
Test MRR: 0.5066356706883463


In [26]:
#For space 6:

ranks6 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos6):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        tr_ranks_avg[j,:] = tr_ranks_avg[j,:] + ranks

        rank = ranks[j] + 1
        ranks6.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "train processed.")

ranks6 = np.array(ranks6)

print()
print("Training Mean rank:", np.mean(ranks6))
print("Hits at 1:", np.mean(ranks6 <= 1))
print("Hits at 10:", np.mean(ranks6 <= 10))
print("Hits at 100:", np.mean(ranks6 <= 100))
print("Hits at 500:", np.mean(ranks6 <= 500))
print("Hits at 1000:", np.mean(ranks6 <= 1000))

print("Training MRR:", np.mean(1/ranks6))

1000 train processed.
2000 train processed.
3000 train processed.
4000 train processed.
5000 train processed.
6000 train processed.
7000 train processed.
8000 train processed.
9000 train processed.
10000 train processed.
11000 train processed.
12000 train processed.
13000 train processed.
14000 train processed.
15000 train processed.
16000 train processed.
17000 train processed.
18000 train processed.
19000 train processed.
20000 train processed.
21000 train processed.
22000 train processed.
23000 train processed.
24000 train processed.
25000 train processed.
26000 train processed.

Training Mean rank: 3.569183580733111
Hits at 1: 0.4374810663435323
Hits at 10: 0.939374431990306
Hits at 100: 0.999924265374129
Hits at 500: 1.0
Hits at 1000: 1.0
Training MRR: 0.6093118704602868


In [27]:

ranks_val6 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_val6):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        val_avg_ranks[j,:] = val_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_val] + 1
        ranks_val6.append(rank)


        j += 1
        if (j) % 1000 == 0: print((j), "val processed.")


ranks_val6 = np.array(ranks_val6)

print()
print("Val Mean rank:", np.mean(ranks_val6))
print("Hits at 1:", np.mean(ranks_val6 <= 1))
print("Hits at 10:", np.mean(ranks_val6 <= 10))
print("Hits at 100:", np.mean(ranks_val6 <= 100))
print("Hits at 500:", np.mean(ranks_val6 <= 500))
print("Hits at 1000:", np.mean(ranks_val6 <= 1000))

print("Validation MRR:", np.mean(1/ranks_val6))

1000 val processed.
2000 val processed.
3000 val processed.

Val Mean rank: 31.196304150257497
Hits at 1: 0.35019691002726444
Hits at 10: 0.8309603150560436
Hits at 100: 0.9697061496516207
Hits at 500: 0.9906089063920024
Hits at 1000: 0.9942441684338079
Validation MRR: 0.5077238195493112


In [28]:

ranks_test6 = []
j = 0 #keep track of all loops
for i, emb in enumerate(text_chem_cos_test6):
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k,:])[::-1]
        ranks = np.argsort(cid_locs)

        test_avg_ranks[j,:] = test_avg_ranks[j,:] + ranks

        rank = ranks[j+offset_test] + 1
        ranks_test6.append(rank)

        j += 1
        if (j) % 1000 == 0: print((j), "test processed.")

ranks_test6 = np.array(ranks_test6)

print()
print("Test Mean rank:", np.mean(ranks_test6))
print("Hits at 1:", np.mean(ranks_test6 <= 1))
print("Hits at 10:", np.mean(ranks_test6 <= 10))
print("Hits at 100:", np.mean(ranks_test6 <= 100))
print("Hits at 500:", np.mean(ranks_test6 <= 500))
print("Hits at 1000:", np.mean(ranks_test6 <= 1000))

print("Test MRR:", np.mean(1/ranks_test6))

1000 test processed.
2000 test processed.
3000 test processed.

Test Mean rank: 22.836413208118753
Hits at 1: 0.3392911239018479
Hits at 10: 0.833383823083914
Hits at 100: 0.9763707967282642
Hits at 500: 0.9936382914268403
Hits at 1000: 0.9960617994547107
Test MRR: 0.5037537684179033


#Rerank from sum

In [29]:

sorted = np.argsort(tr_ranks_avg)
new_tr_ranks = np.diag(np.argsort(sorted)) + 1

print(np.mean(new_tr_ranks))
print("%1:", np.mean(new_tr_ranks <= 1))
print("%10:", np.mean(new_tr_ranks <= 10))
print("%100:", np.mean(new_tr_ranks <= 100))

print("Trainng MRR:", np.mean(1/np.array(new_tr_ranks)))


2.582777946076946
%1: 0.5575583156619206
%10: 0.9703120266585883
%100: 1.0
Trainng MRR: 0.708569958676293


In [30]:


sorted = np.argsort(val_avg_ranks)
val_final_ranks = np.argsort(sorted) + 1
new_val_ranks = np.diag(val_final_ranks[:,offset_val:offset_test])

print(np.mean(new_val_ranks))
print("%1:", np.mean(new_val_ranks <= 1))
print("%10:", np.mean(new_val_ranks <= 10))
print("%100:", np.mean(new_val_ranks <= 100))

print("Validation MRR:", np.mean(1/np.array(new_val_ranks)))


17.837624962132686
%1: 0.41805513480763407
%10: 0.884580430172675
%100: 0.9781884277491669
Validation MRR: 0.5797269257879241


In [31]:

sorted = np.argsort(test_avg_ranks)
test_final_ranks = np.argsort(sorted) + 1
new_test_ranks = np.diag(test_final_ranks[:,offset_test:])

print(np.mean(new_test_ranks))
print("%1:", np.mean(new_test_ranks <= 1))
print("%10:", np.mean(new_test_ranks <= 10))
print("%100:", np.mean(new_test_ranks <= 100))

print("Test MRR:", np.mean(1/new_test_ranks))


17.77158436837322
%1: 0.4359285065131778
%10: 0.8900333232353832
%100: 0.9851560133292941
Test MRR: 0.5917850369537417
